<a href="https://colab.research.google.com/github/marantmir/python_cases/blob/master/DataScience_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **More Academy**
## Data Science do Zero com Python

### O que é Data Science?

Para além da programação e das demais ferramentas, Ciência de Dados é um campo interdisciplinar e desta forma envolve a aplicação de outras disciplinas como Matemática e Estatística, com ênfase na análise de dados para possíveis tomadas de decisão.
Assim, o objetivo desta trilha é demonstrar os principais conceitos estatísticos a serem utilizados em Ciência de Dados. Não pretende ser um resumo exaustivo ou definitivo, visto a complexidade destas áreas.

<img src = 'https://abracd.org/wp-content/uploads/2019/04/diagrama-data-science.png'/>

## Ecossitema do Python

### Eixo base

1. NumPy

Biblioteca de manipulação de arrays multidimensionais de alta performance, possui diversas funções matemáticas e estatísticas.

2. Pandas

Principal biblioteca de manipulação e análise de dados. Possui diversos métodos para agrupar, filtrar, manipular, limpar e combinar dados de forma simples.

3. StatsModels

Módulo com diversas funções estatísticas.

4. SciPy

Biblioteca para computação científica, com funções de álgebra linear, cálculo, estatística, dentre outras.

### Data Visualization

1. Matplotlib

2. Seaborn

3. Plotly

4. Altair

5. Bokeh

6. Pydot

7. Yellowbrick

### Machine Learning e Deep Learnin

1. Scikit-learn

2. TensorFlow

3. PyTorch

4. Keras


### Scraping

1. Scrapy

2. Beautiful Soup

### Processamento de Linguagem Natural

1. NLTK

2. SpaCy

3. Gensim

Há diversas outras bibliotecas e módulos dentro do ecossitema Pythônico. 

## Hands On!

Neste hands on vamos fazer uma análise exploratória com microdados do ENEM. Nosso objetivo será apresentar os principais comandos da biblioteca Pandas e inserir técnicas de AutoEDA e DataViz.

Sugerimos o Google Colab como ambiente de desenvolvimento.

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
import sweetviz as sv
import matplotlib.pyplot as plt
import seaborn as sns

#### Carrega o dataset e obtém informações iniciais

In [ ]:
# carrega o dataset
df_enem = pd.read_csv('enem_sample_data.csv')

In [ ]:
# mostra as 5 primeiras linhas do dataset
df_enem.head()

In [ ]:
# mostra as 5 últimas linhas do dataset
df_enem.tail()

In [ ]:
# quantidade de linhas e colunas
df_enem.shape

In [ ]:
# array com os nomes das colunas
df_enem.columns

In [ ]:
# informações gerais 
# outra possibilidade para verificar o dtype de cada coluna: df.dtypes
df_enem.info()

In [ ]:
# quantidade de valores únicos por cada coluna
df_enem.nunique()

In [ ]:
# verifica percentual de missing values para cada coluna
((df_enem.isna().sum()/df_enem.shape[0])*100).sort_values(ascending = False)

#### Tratamento e limpeza dos dados

Vamos renomear algumas colunas. Mas, antes iremos criar uma cópia do banco de dados original.

In [ ]:
df = df_enem.copy()

In [ ]:
new_cols = {
    'NO_MUNICIPIO_RESIDENCIA': 'municipio_residencia',
    'NO_MUNICIPIO_PROVA': 'municipio_prova',
    'NU_IDADE': 'idade',
    'TP_SEXO': 'sexo',
    'TP_ESTADO_CIVIL': 'estado_civil',
    'TP_COR_RACA': 'cor_raca',
    'IN_TREINEIRO': 'treineiro',
    'NU_NOTA_CN': 'ciencias_natureza',
    'NU_NOTA_CH': 'ciencias_humanas',
    'NU_NOTA_LC': 'linguagens',
    'NU_NOTA_MT': 'matematica',
    'NU_NOTA_REDACAO': 'redacao',
    'Q001': 'escolaridade_pai',
    'Q002': 'escolaridade_mae',
    'Q006': 'renda_familiar_mensal',
    'Q025 ': 'internet'
}

df.rename(columns = new_cols, inplace = True)

Vamos verificar se há linhas duplicadas:

In [ ]:
# como vemos não há linhas duplicadas
df[df.duplicated()]

In [ ]:
# outra possibilidade seria:
df.duplicated().sum()

Caso houvesse linhas duplicadas poderíamos usar a função `.drop_duplicates()` para corrigir este problema.

Vamos remover as colunas `NU_INSCRICAO` e `NO_MUNICIPIO_NASCIMENTO`.

In [ ]:
cols_to_del = [' NU_INSCRICAO', 'NO_MUNICIPIO_NASCIMENTO']
df.drop(columns = cols_to_del, inplace = True)

Por fim, quanto aos dados ausentes iremos apenas remover, mas há diversas técnicas para preenchimento de missing data que não teremos tempo de abordar aqui.

In [ ]:
provas = df.columns.to_list()[7:12]

In [ ]:
df.dropna(subset = provas, inplace = True)

In [ ]:
df.isna().sum()

Agora precisamos detectar alguma inconsistência com os valores dos campos numéricos.

In [ ]:
df.describe().T

In [ ]:
# vamos criar uma coluna com a média global dos alunos
df['media'] = df[provas].mean(axis = 1)

In [ ]:
# por fim, vamos ordenar os dados pelos alunos com maior média global
df.sort_values(by = ['media'], ascending = False, inplace = True)

Alteremos os valores dos campos `sexo`,`estado_civil` e `cor_raca`.

In [ ]:
df['sexo'] = df['sexo'].map({'M':'Masculino',
                'F':'Feminino'
                    })

Agora vamos utilizar uma forma distinta para resolver este mesmo problema, usando funções.

In [ ]:
def editar_cor_raca(cor_raca):
    if cor_raca == 0:
        return 'Não informado'
    elif cor_raca == 1:
        return 'Branca'
    elif cor_raca == 2:
        return 'Preta'
    elif cor_raca == 3:
        return 'Parda'
    elif cor_raca == 4:
        return 'Amarela'
    else:
        return 'Indígena'
    
def editar_estado_civil(estado_civil):
    if estado_civil==0:
        return 'Não Informado'
    elif estado_civil==1:
        return 'Solteiro (a)'
    elif estado_civil==2:
        return 'Casado (a)'
    elif estado_civil==3:
        return 'Divorciado (a)'
    else:
        return 'Viúvo (a)'

In [ ]:
df['estado_civil'] = df['estado_civil'].apply(editar_estado_civil)
df['cor_raca'] = df['cor_raca'].apply(editar_cor_raca)

In [ ]:
# verificar os valores únicos da coluna após a transformação
df['estado_civil'].unique()

In [ ]:
df['cor_raca'].unique()

In [ ]:
df.head()

### AutoEDA
#### Pandas Profiling

In [ ]:
eda_profile = ProfileReport(df, 
                            title = 'EDA ENEM', 
                            html = {'style':{'full_width':True}})

In [ ]:
# mostra o relatório
eda_profile.to_notebook_iframe()

In [ ]:
# salva o relatório
eda_profile.to_file(output_file="dataframe_report.html")

#### SweetViz

In [ ]:
# !pip install sweetviz

In [ ]:
eda_profile = sv.analyze(df, 'matematica')
eda_profile.show_html('dataset.html')

#### Consultas

#### Data Visualization

Nesta seção iremos explorar algumas possibilidades de visualização dos dados utilizando `matplotlib` e `seaborn`.

##### Perfil geral dos inscritos

In [ ]:
df['municipio_residencia'].value_counts()[:20]

In [ ]:
lista_municipios = df['municipio_residencia'].value_counts()[:20]
fig, ax = plt.subplots(figsize = (12, 5))
ax.barh(
    width = lista_municipios.values,
    y = lista_municipios.index
)
ax.set_yticklabels(labels = lista_municipios.index,rotation = 45)
ax.set(
    xlabel = 'Quantidade de inscritos',
    ylabel = 'Municípios',
    
)
ax.set_title(label = 'Residência dos inscritos no ENEM.', 
             pad = 10,
             fontsize = 14, 
             fontweight = 'bold',
             color = 'gray'
            );
#ax.grid(axis = 'y', alpha = 0.2)

In [ ]:
fig, ax = plt.subplots(figsize = (8, 4))
ax.hist(df['idade'], bins = 30,
        color = 'gray'
       )
ax.set(xlabel = 'Idade', ylabel = 'Frequência')
ax.set_title('Perfil geral dos inscritos no ENEM', fontsize = 12, color = 'gray');

In [ ]:
# outra possibilidade: df['idade'].plot(kind = 'hist');

##### Perfil geral das notas

In [ ]:
# estatística descritiva das provas
df[provas].describe()

In [ ]:
df[provas].plot(kind = 'box');

In [ ]:
fig, ax = plt.subplots(figsize = (10, 5))
sns.boxplot(data = df[provas], ax = ax, 
            showfliers = True,
            showmeans = True
           )
ax.set_xticklabels(['Ciências da Natureza', 'Ciências Humanas', 'Linguagens', 'Matemática', 'Redação'])
ax.set(xlabel = 'Provas')
ax.set_title('Boxplot das notas dos inscritos no ENEM', color = 'gray', size = 12);

In [ ]:
sns.pairplot(df[provas+['sexo']], hue = 'sexo')

In [ ]:
for prova in provas:
    sns.displot(df, x = prova, hue = 'sexo')

In [ ]:
filtro = df['escolaridade_pai'].values
filtro.sort()
fig, ax = plt.subplots(figsize = (8, 5))
sns.boxplot(data = df, 
            x = filtro,
            y = df['matematica'], ax = ax)
ax.set(xlabel = 'Escolaridade do pai',
       ylabel = 'Nota em matemática'
      )
ax.set_title('Nota em matemática por grau de escolaridade do pai', fontsize = 12, color = 'gray');

In [ ]:
renda_familiar = df['renda_familiar_mensal'].values
renda_familiar.sort()
fig, ax = plt.subplots(figsize = (8, 5))
sns.boxplot(data = df, 
            x = renda_familiar,
            y = df['matematica'], ax = ax)
ax.set(xlabel = 'Níveis de renda familiar',
       ylabel = 'Nota em matemática'
      )
ax.set_title('Nota em matemática por renda familiar', fontsize = 12, color = 'gray');

In [ ]:
corr = df[provas].corr()
sns.heatmap(corr, annot = True, cmap = 'Blues');

#### Aplicando Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
# variável target
y = df['matematica']  
# matriz com as variáveis explicativas
variaveis_categoricas = pd.get_dummies(df[['sexo', 'cor_raca', 'escolaridade_pai', 'internet', 'renda_familiar_mensal']], 
               drop_first = True
              )
variaveis_numericas = df['ciencias_natureza']
X = pd.concat([variaveis_categoricas, variaveis_numericas], axis = 1)

In [ ]:
# separa os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
# instancia e treina o modelo

In [ ]:
linear_regression = LinearRegression()

In [ ]:
linear_regression.fit(X_train, y_train)

In [ ]:
# score
linear_regression.score(X_test, y_test)

In [ ]:
# coeficientes estimados
linear_regression.coef_

In [ ]:
y_pred = linear_regression.predict(X_test)

In [ ]:
# calcula o MSE
# quanto menor, melhor o modelo
mean_squared_error(y_test, y_pred)